In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, roc_auc_score 
from sklearn import svm

In [2]:
def status_map(status):
    if 'no_relapse' in status or 'NoRelapse' in status:
        return 0
    else:
        return 1

status_list =['relapse', 'no_relapse', 
              'test1relapse', 'test1no_relapse', 
              'test2relapse', 'test2no_relapse', 
              'NewTest1_Relapse', 'NewTest1_NoRelapse', 
              'NewTest2_Relapse', 'NewTest2_NoRelapse']

class BcData:
    def __init__(self):
        self.data = pd.read_csv("data/data_good.csv")
        self.total = pd.read_csv("data/Total_old.csv", names=["gsm", "status"])
        self._drop_grey()
        self._log_table()

    # Drop grey columns
    def _drop_grey(self):
        self.gsm_series = self.total[self.total.status.isin(status_list)].gsm
        new_cols = pd.Series(["GeneSymbol"]).append(self.gsm_series)

        self.total = self.total[self.total.gsm.isin(self.gsm_series)]
        self.data = self.data.filter(items=new_cols)

    # Group rows by gene leaving max median row
    def _groupby_gene(self):
        return self.data\
            .groupby("GeneSymbol", as_index=False, sort=False)\
            .apply(lambda f: f.loc[f.median(axis=1).idxmax()])

    def _log_table(self):
        index = self.data.iloc[:, 0]
        self.data = np.log2(self.data.iloc[:, 1:])
        self.data.insert(0, "GeneSymbol", index)

    def _get_status(self):
        return self.total.status.map(status_map)

    # Drop rows with quantile less than threshold (values = {7, 8, 9})
    def filter_percentile(self, quantile=1, threshold=9):
        q = self.data.quantile(q=quantile, axis=1)
        index = q[q >= threshold].index.values
        self.data = self.data.loc[index, :]

    # Drop rows with max/min diff less than threshold (values = {1.5, 2})
    def filter_diff_percentile(self, qmax=1, qmin=0, threshold=2):
        threshold = np.log2(threshold)
        max = self.data.quantile(q=qmax, axis=1)
        min = self.data.quantile(q=qmin, axis=1)
        index = max[max - min >= threshold].index.values
        self.data = self.data.loc[index, :]

    # Final mutation
    def group_data(self):
        grouped = self._groupby_gene().T
        self.data = grouped.rename(columns=grouped.iloc[0]).drop(grouped.index[0])
        
    # values = {0, 1, 2, 3, 4}
    def train_test_split(self, test_fold_number=0):
        test_status = status_list[test_fold_number:test_fold_number+2]
        total_test = self.total[self.total.status.isin(test_status)]
        total_train = self.total[~self.total.status.isin(test_status)]
        
        X_test = self.data.loc[total_test.gsm]
        X_train = self.data.loc[total_train.gsm]
        y_test = total_test.status.map(status_map)
        y_train = total_train.status.map(status_map)
        
        return X_train, X_test, y_train, y_test

In [3]:
bc = BcData()
bc.filter_diff_percentile(qmax=0.75, qmin=0.25, threshold=1.5)
bc.filter_percentile(quantile=0.75, threshold=7)
bc.group_data()
bc.data

,STAT1,GAPDH,ACTB,PRPF8,GDI2,RPL21,EIF3F,RPS5,RPL10A,RPL17,...,TMEM135,LASS2,MRPL17,SLC27A3,ACTR10,LRRC59,ISYNA1,UBQLN4,SH3BP4,KCNE4
GSM441628,10.1189,12.4899,12.3241,8.12817,10.2657,12.6354,10.2893,12.3643,12.0118,12.9817,...,7.4559,10.3364,9.39708,8.1336,10.2042,9.54129,7.41549,6.92487,9.61767,5.0048
GSM441629,9.45188,11.89,11.1485,9.80213,8.99319,11.964,10.0725,11.6801,11.5113,12.2222,...,7.36097,10.5038,8.98289,7.57143,9.32403,10.963,7.00116,7.44613,7.91139,8.47747
GSM441643,8.3548,12.8534,12.2794,9.26371,9.68239,12.0003,10.9454,12.4005,11.3411,12.486,...,6.02202,10.8261,9.55747,7.52877,9.72201,9.55876,8.22317,7.19559,9.03323,7.93546
GSM441644,10.6103,13.2989,12.1458,8.18223,10.006,12.4623,10.0924,12.8321,11.9007,12.7933,...,7.59432,9.8764,9.36144,5.99566,10.2223,9.32937,7.16173,6.67637,8.4719,5.65332
GSM441657,10.7411,12.6399,12.1225,8.69317,9.86313,12.6431,10.5805,12.3452,12.3039,12.8404,...,8.68881,10.9884,8.70488,6.36779,9.8479,9.67044,6.74766,6.58517,10.1066,6.98088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM79256,8.951,11.7398,12.2444,9.204,8.56978,12.2456,10.312,11.9125,11.9337,12.5949,...,6.55152,10.3828,7.52995,6.86207,9.2944,8.40215,7.08712,7.0161,9.45736,5.83043
GSM79307,8.63579,11.874,12.7701,8.5807,8.93809,11.966,10.0928,11.523,11.8744,12.4581,...,7.40512,10.2444,7.59653,7.25855,9.25592,8.7403,7.90226,7.1078,9.03438,6.49406
GSM79194,8.23052,11.6573,12.8212,9.41964,8.54347,11.4439,9.82976,11.2178,10.87,12.0338,...,6.00599,10.3581,7.38679,7.42136,8.55693,8.91121,7.54922,6.41859,9.14434,7.35013
GSM79179,8.70025,12.5266,12.5553,8.44396,8.8949,12.1446,10.2204,11.1942,12.0194,12.0932,...,6.62486,10.6735,8.18407,7.44046,9.39059,8.90865,7.00536,7.23688,10.008,7.48382


In [10]:
X_train, X_test, y_train, y_test = bc.train_test_split(test_fold_number=2)

# Set dual = True if number of features > number of examples and vice versa
# clf = svm.LinearSVC(penalty='l1', dual=False, C=1, max_iter=100000)
clf  = svm.SVC(kernel="linear", C=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

print("sensitivity = {} auc = {}".format(recall, auc))

sensitivity = 0.5833333333333334 auc = 0.6871890547263683


In [22]:
step = 50
total = len(X_train.columns)

# for i in range(0, total, step):
for i in range(total):
    if i + step >= total:
        break
        
    clf  = svm.SVC(kernel="linear", C=1)
    clf.fit(X_train.iloc[:, i:i+step], y_train)

    y_pred = clf.predict(X_test.iloc[:, i:i+step])
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    if (recall > 0.6 and auc > 0.65):
        print("For {} step: sensitivity = {} auc = {}".format(i, recall, auc))    

For 8 step: sensitivity = 0.6666666666666666 auc = 0.7213930348258706
For 9 step: sensitivity = 0.6666666666666666 auc = 0.7288557213930348
For 15 step: sensitivity = 0.75 auc = 0.7854477611940298
For 16 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 17 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 18 step: sensitivity = 0.75 auc = 0.7779850746268656
For 19 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 20 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 21 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 22 step: sensitivity = 0.6666666666666666 auc = 0.736318407960199
For 32 step: sensitivity = 0.6666666666666666 auc = 0.7587064676616915
For 160 step: sensitivity = 0.6666666666666666 auc = 0.7288557213930348
For 193 step: sensitivity = 0.6666666666666666 auc = 0.7288557213930348
For 194 step: sensitivity = 0.6666666666666666 auc = 0.7064676616915422
For 195 step: sensitivity = 0.6666

For 2129 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 2130 step: sensitivity = 0.6666666666666666 auc = 0.6990049751243781
For 2131 step: sensitivity = 0.6666666666666666 auc = 0.7213930348258706
For 2132 step: sensitivity = 0.6666666666666666 auc = 0.7512437810945273
For 2133 step: sensitivity = 0.6666666666666666 auc = 0.7512437810945273
For 2186 step: sensitivity = 0.6666666666666666 auc = 0.7661691542288557
For 2199 step: sensitivity = 0.6666666666666666 auc = 0.7437810945273631
For 2201 step: sensitivity = 0.75 auc = 0.8078358208955224
For 2206 step: sensitivity = 0.6666666666666666 auc = 0.7661691542288557
For 2348 step: sensitivity = 0.6666666666666666 auc = 0.7064676616915422
For 2349 step: sensitivity = 0.6666666666666666 auc = 0.7213930348258706
For 2368 step: sensitivity = 0.6666666666666666 auc = 0.7139303482587064
For 2698 step: sensitivity = 0.75 auc = 0.7555970149253731
For 2699 step: sensitivity = 0.6666666666666666 auc = 0.7139303482587064
For 28